In [1]:
# Import library-library
import os
import glob
import string

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
excluded_words = ["tempat", "waktu", "mencuri", "rampok"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessing(berita):
    # Preprocessing
    s = berita.lower()
    s = s.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    s = s.encode('ascii', 'replace').decode('ascii')
    ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", s).split())
    s.replace("http://", " ").replace("https://", " ")
    s = re.sub('\s+', ' ', s)
    s = s.strip()
    s = s.translate(str.maketrans("","", string.punctuation))
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'\b\w{1,1}\b', '', s)
    tokens = [token for token in s.split(" ") if token != ""]

    # Stopwords checking
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [4]:
df =pd.read_csv("scrapped_news/scrapped_news.csv")
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1342 entries, 0 to 1341
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1342 non-null   object
 1   date         1342 non-null   object
 2   description  1342 non-null   object
 3   source       1342 non-null   object
dtypes: object(4)
memory usage: 42.1+ KB


In [5]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df = pd.read_csv("scrapped_news/scrapped_news.csv")
df = df.dropna(subset=['description'], axis=0)
df.sort_values("description", inplace = True)
df.drop_duplicates(subset ="description", keep = False, inplace = True)

df.head()
df_train_unprocessed, df_test_unprocessed = train_test_split(df,test_size=0.1)

In [6]:
for x in range(0, df.shape[0]):
    text = preprocessing(df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [7]:
df.head()
df_train, df_test = train_test_split(df,test_size=0.1)
df_train.head()

,title,date,description,source
1219,Gawat Kali! Pria Bermotor Terekam Curi Beras Z...,"Sabtu, 30 Apr 2022 14:18 WIB",pria sepeda motor terekam cctv diduga mencuri ...,www.detik.com
323,"Modus Kesurupan, Pria Bulungan Perkosa Anak Ti...","Rabu, 20 Apr 2022 03:29 WIB",pria inisial hp 41 kabupaten bulungan kalimant...,www.detik.com
1087,"Dikepung Polisi, Staf Kampus STKIP Bima Pemerk...","Kamis, 16 Jun 2022 15:12 WIB",staf kampus stkip bima ditangkap polisi terkai...,www.detik.com
531,"Pelajar di Luwu Diperkosa di Indekos, Pelaku D...","Jumat, 15 Apr 2022 03:28 WIB",pelajar inisial wn 13 kabupaten luwu sulawesi ...,www.detik.com
463,Pria Ditembak di Sumenep Diduga Begal,"Minggu, 13 Mar 2022 21:55 WIB",video menampilkan pria ditembak beredar aplika...,www.detik.com


In [8]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, "corpus/model/desc_text_test_unprocessed.pkl")

['corpus/model/desc_text_test_unprocessed.pkl']

In [9]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [10]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  135
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  1207


In [11]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total.csv', index=False)
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [12]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(135, 4939)
tfidf_train_matrix
(1207, 14708)


In [13]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer.pkl")
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train.pkl")

['corpus/matrix/tfidf_train.pkl']

### Pencarian 

In [14]:
df_total = pd.read_csv('corpus/dataset/df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1342 entries, 0 to 1341
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1342 non-null   object
 1   date         1342 non-null   object
 2   description  1342 non-null   object
 3   source       1342 non-null   object
dtypes: object(4)
memory usage: 52.4+ KB
None
------------------------------------------------------------------------------------------
135
1207


In [15]:
kueri='curi'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['curi']


Top 10 most similar documents in corpus:
No ID Dokumen  :  835
Tanggal        :  Selasa, 17 Mei  2022 07:10 WIB
Isi berita     :  pria berinisial alias aden 30 diduga membunuh wanita sukabumi eneng kulsum 35 kabur ditangkap polisi pelariannya aden menyamar wig peci dilansir selasa 1752022 lelaki disebutsebut mantan kekasih korban memakai peci biru rambut palsu terurai pelaku kabur daerah parungkuda selesai menusuk korban tersangka kabur wig kopiah peci menyamarkan pelarian pelaku lari daerah parungkuda bermalam daerah cibenda karang kasat reskrim polres sukabumi akp putu hasti hermawan keberadaan pelaku terlacak tim gabungan satreskrim polres sukabumi unit reskrim polsek cibadak mengantongi informasi saksi informasi keberadaan pelaku gubuk tim pengejaran berhasil menangkap pelaku posisinya mengarah gunung walat kabupaten sukabumi putu pembunuhan menggegerkan warga cibadak kabupaten sukabumi wanita tewas ditusuk pria rumahnya jumat 145 malam 2000 wib korban bernama eneng kuls